# Sentinel-2 multiclass classification using Categorical Boosting

In [ ]:
!pip install catboost

## Download the EuroSAT dataset


[EuroSAT dataset](https://github.com/phelber/EuroSAT)
![](https://raw.githubusercontent.com/phelber/EuroSAT/master/eurosat_overview_small.jpg)


Download the dataset

In [ ]:
import requests

url =  'http://madm.dfki.de/files/sentinel/EuroSAT.zip'
r = requests.get(url, allow_redirects=True)
open('EuroSAT.zip', 'wb').write(r.content)

Unzip the dataset

In [ ]:
!unzip '/content/EuroSAT.zip'

Rename the folder

In [ ]:
!mv '/content/2750' '/content/EuroSAT'

Count images in the dataset

In [ ]:
import glob

paths = glob.glob('/content/EuroSAT/*/*')
print('Number of images {}/27000'.format(len(paths)))

## Image Data Generator 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2)
      
train_generator = train_datagen.flow_from_directory(
    '/content/EuroSAT',  
    target_size=(10,10),  
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

val_generator = train_datagen.flow_from_directory(
    '/content/EuroSAT',  
    target_size=(10,10),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')  

In [ ]:
classes = train_generator.class_indices
classes= dict(zip(classes.values(), classes.keys()))  
print(classes)

Generate some exaples

In [ ]:
x,y = next(iter(train_generator))
print('x shape {}'.format(x.shape))
print('y shape {}'.format(y.shape))

Plot some examples

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(nrows = 4, ncols = 4, figsize = (16,16))

ct = 0
for i in range(4):
  for j in range(4):
    axes[i,j].imshow(x[ct])
    axes[i,j].set_title('Label {} \n Class {}'.format(y[ct], classes[np.argmax(y[ct])]))
    axes[i,j].axis(False)
    ct += 1

fig.tight_layout()
plt.show()


## Define the Machine Learning model

We will use the CatBoost ([Find out more](https://catboost.ai/en/docs/))

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=5000,
                          learning_rate=0.02,
                          max_depth=4,
                          loss_function='MultiClass',
                          eval_metric='Accuracy',
                          early_stopping_rounds = 30,
                          verbose = 50)

## Train the CatBoostClassifier

In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/content/EuroSAT',  
    target_size=(10, 10),  
    batch_size=train_generator.n,
    class_mode='categorical',
    subset='training')

val_generator = train_datagen.flow_from_directory(
    '/content/EuroSAT',  
    target_size=(10, 10),
    batch_size=val_generator.n,
    class_mode='categorical',
    subset='validation')  


x_train,y_train = next(iter(train_generator))
x_val, y_val = next(iter(train_generator))

In [ ]:
from catboost import Pool

y_train = np.argmax(y_train, axis=1)
y_val = np.argmax(y_val, axis=1)

train_pol = Pool(x_train.reshape((x_train.shape[0], x_train.shape[1]*x_train.shape[2]*x_train.shape[3])),y_train)
val_pol = Pool(x_val.reshape((x_val.shape[0], x_val.shape[1]*x_val.shape[2]*x_val.shape[3])), y_val)

In [ ]:
model.fit(train_pol, eval_set=val_pol)

## Evaluate Performances

In [ ]:
y_pred = model.predict(x_val.reshape((x_val.shape[0], x_val.shape[1]*x_val.shape[2]*x_val.shape[3])))

Compute confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_val, y_pred, normalize='true')

Plot confusion matrix

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10,8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes.values())
disp.plot(ax=ax, cmap=plt.cm.Blues, xticks_rotation='vertical')
plt.show()

Compute classification report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred, target_names=classes.values()))